In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import plotly.express as px
import warnings
import matplotlib.colors as mcolors
from beziers import *
from keras import layers,models
import tensorflow as tf
from sklearn.model_selection import train_test_split
from joblib import Parallel,delayed
import csv
from tensorflow.keras.callbacks import ModelCheckpoint
from scipy.spatial.distance import pdist, squareform
import pickle
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.saving import register_keras_serializable
import umap
from sklearn.metrics import silhouette_score
from math import ceil


KeyboardInterrupt: 

In [ ]:
#INITIALISER ET CHARGER LES DONNEES DE DEPART   
col_lon = "LONGITUDE"
col_lat = "LATITUDE"
col_alti = "ALTI_STD_FT"
column_names = [col_lon,col_lat]
col_groupby = "SYST_TRAJ_ID"
col_sample = "SYST_POINT_ID"

df = pd.read_parquet('samples.parquet')


In [ ]:
#ECHANTILLONNE LES DONNEES PAR LONGUEUR DE CORDE

def resample_df_by_curve_length(df, col_groupby, col_sample, column_names, number_point=99):
    datas = {name: [] for name in [col_groupby, col_sample] + column_names}
    groups = df.groupby(col_groupby,sort=True)
    
    for i, (group, f) in enumerate(groups):
        
        # Calculer la longueur cumulative le long de la courbe
        f = f.sort_values(by=col_sample).reset_index(drop=True)
        distances = np.sqrt(sum((f[col].diff() ** 2) for col in column_names))
        cumulative_length = distances.cumsum()
        # Normaliser la longueur cumulative entre 0 et 100
        cumulative_length = 100 * (cumulative_length - cumulative_length.min()) / (cumulative_length.max() - cumulative_length.min())
        f['cumulative_length'] = cumulative_length
        
        # Créer des points rééchantillonnés équidistants le long de la longueur normalisée
        r = np.linspace(0, 100, number_point + 1)
        
        # Ajouter les points de groupe et d'échantillon aux données
        datas[col_groupby] += [group] * len(r)
        datas[col_sample] += r.tolist()
        
        # Interpoler chaque colonne pour correspondre aux nouvelles valeurs de longueur
        for column_name in column_names:
            interp = interp1d(f['cumulative_length'], f[column_name], fill_value="extrapolate")
            interpolated_values = list(interp(r).flatten())
            datas[column_name] += interpolated_values
    
    return pd.DataFrame(datas)
resampled_df_curve_length = resample_df_by_curve_length(df, col_groupby, col_sample, column_names)
resampled_df_curve_length = resampled_df_curve_length.sort_values(by=col_sample).reset_index(drop=True)
resampled_df_curve_length[column_names] = resampled_df_curve_length[column_names].interpolate(method='linear')

In [ ]:
#PERMET DE VISUALISER LES DIRECTIONS DES TRAJECTOIRES

def plot_trajs(df, col_groupby,col_order, col_x, col_y, num_traj_max=10000):
    plt.figure(dpi=200)
    for i,(traj_id, traj) in enumerate(df.groupby(col_groupby)):
        n_colors=len(traj)*2
        cmap = mcolors.LinearSegmentedColormap.from_list("red_blue", ["#FF0000", "#0000FF"], N=n_colors)
        traj = traj.sort_values(by=col_order)
        colors = [cmap(j / (n_colors-1)) for j in range(0,n_colors,2)]
        for j in range(len(traj)-1):
            plt.plot(traj[col_x].iloc[j:j+2],traj[col_y].iloc[j:j+2],lw=0.5,color=colors[j] )
            if i>=num_traj_max-1:
                break
    plt.xlabel(col_x)
    plt.ylabel(col_y)
    plt.axis("equal")
    plt.show()
plot_trajs(df,col_groupby,col_order=col_sample,col_x=col_lon,col_y=col_lat)

In [ ]:
#PERMET DE VISUALISER INDIVIDUELLEMENT LES TRAJECTOIRES
def plot_trajs2(df, col_groupby, col_order, col_x, col_y, selected_traj=None, num_traj_max=10000):
    plt.figure(dpi=200)

    # Obtenir toutes les trajectoires groupées
    grouped_trajs = list(df.groupby(col_groupby))
    
    # Si une trajectoire spécifique est sélectionnée
    if selected_traj is not None:
        # Vérifier que l'indice est dans les limites
        if selected_traj < len(grouped_trajs):
            # Extraire la trajectoire correspondant à l'ordre
            traj_id, traj = grouped_trajs[selected_traj]
            
            # Traiter et tracer cette trajectoire
            n_colors = len(traj) * 2
            cmap = mcolors.LinearSegmentedColormap.from_list("red_blue", ["#FF0000", "#0000FF"], N=n_colors)
            traj = traj.sort_values(by=col_order)
            colors = [cmap(j / (n_colors - 1)) for j in range(0, n_colors, 2)]
            for j in range(len(traj) - 1):
                plt.plot(traj[col_x].iloc[j:j + 2], traj[col_y].iloc[j:j + 2], lw=0.5, color=colors[j])
        else:
            print(f"Indice de trajectoire {selected_traj} hors limites. Nombre total de trajectoires : {len(grouped_trajs)}")
    else:
        # Sinon, tracer toutes les trajectoires jusqu'à `num_traj_max`
        for i, (traj_id, traj) in enumerate(grouped_trajs):
            n_colors = len(traj) * 2
            cmap = mcolors.LinearSegmentedColormap.from_list("red_blue", ["#FF0000", "#0000FF"], N=n_colors)
            traj = traj.sort_values(by=col_order)
            colors = [cmap(j / (n_colors - 1)) for j in range(0, n_colors, 2)]
            for j in range(len(traj) - 1):
                plt.plot(traj[col_x].iloc[j:j + 2], traj[col_y].iloc[j:j + 2], lw=0.5, color=colors[j])
            if i >= num_traj_max - 1:
                break

    plt.xlabel(col_x)
    plt.ylabel(col_y)
    plt.axis("equal")
    plt.show()
plot_trajs2(df, col_groupby=col_groupby, col_order=col_sample, col_x=col_lon, col_y=col_lat, selected_traj=432)


In [ ]:
#PERMET DE VISUALISER LE RESAMPLING POUR S'ASSURER QU'IL A ETE FAIT CORRECTEMENT

def plot_trajs_points(df,df2, col_groupby,col_order, col_x, col_y, num_traj_max=10000):
    plt.figure(dpi=200)
    for i,(traj_id, traj) in enumerate(df.groupby(col_groupby)):
        traj = traj.sort_values(by=col_order)
        plt.scatter(traj[col_x], traj[col_y],color="b",s=0.5)
        if i>=num_traj_max-1:
            break
    for i,(traj_id, traj) in enumerate(df2.groupby(col_groupby)):
         traj = traj.sort_values(by=col_order)
         plt.scatter(traj[col_x], traj[col_y],color="r",s=0.5,alpha=0.5)
         if i>=num_traj_max-1:
             break
    plt.xlabel(col_x)
    plt.ylabel(col_y)
    plt.axis("equal")
    plt.show()
plot_trajs_points(df,resampled_df_curve_length, col_groupby,col_sample, col_lon, col_lat) 

In [ ]:
#PERMET DE VISUALISER LES TRAJECTOIRES APPROXIMEES PAR LES BEZIERS

def plot_trajs_beziers(df, col_groupby,col_order, col_x, col_y, num_traj_max=1000):
    plt.figure(dpi=200)
    groups = df.groupby(col_groupby)
    for i,(traj_id, traj) in enumerate(groups):
        traj = traj.sort_values(by=col_order)
        points = traj[['LONGITUDE', 'LATITUDE']].to_numpy()
        beziers = fit_curve(points)
        control_points=get_control_points(beziers)
        print(len(control_points))
        for curve in beziers:
            curve = np.array(curve)  
            t_values = np.linspace(0, 1, 100)
            bezier_points = np.array([bezier_q(curve, t) for t in t_values]) 
            print(curve)
            plt.plot(bezier_points[:, 0], bezier_points[:, 1], color="r")
        for cp in control_points:
            plt.scatter(cp[0],cp[1],color='green',s=4)
        plt.plot(traj[col_x], traj[col_y],color="b",alpha=0.5)
        if i>=num_traj_max-1:
            break
    plt.xlabel(col_x)
    plt.ylabel(col_y)
    plt.axis("equal")
    plt.show()
plot_trajs_beziers(resampled_df_curve_length,col_groupby=col_groupby,col_order=col_sample,col_x=col_lon,col_y=col_lat)

In [ ]:
#PERMET DE CALCULER LES POINTS DE CONTROLES DE BEZIERS POUR CHAQUE TRAJECTOIRE ET DES LES SAUVEGARDER
def get_beziers_traj(df, col_groupby, number_traj=1000):
    groups = df.groupby(col_groupby,sort=True)
    selected_groups = groups
    def bezier_from_group(group):
          
        points = group[['LONGITUDE', 'LATITUDE']].to_numpy()
        beziers = fit_curve(points)
        control_points = get_control_points(beziers)
        print(control_points)
        return np.array(control_points).flatten()
    control_points_list=Parallel(n_jobs=-1)(delayed(bezier_from_group)(group) for traj_id, group in selected_groups )
    return control_points_list
cp=get_beziers_traj(resampled_df_curve_length,col_groupby,number_traj=1000)
with open('data10.csv', 'w', newline='') as f:
      writer = csv.writer(f)
      writer.writerows(cp)

In [ ]:
#PERMET DE RECHARGER LES POINTS DE CONTROLES
cp = []
with open('data10.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
         control_points = np.array(row, dtype=float)
         cp.append(control_points)  

In [ ]:
#CREER LE MODELE D'AUTOENCODER, ENCODER ET DECODER
def create_autoencoder(input_dim, pregu=0.0001, perte='mean_squared_error', n=1, learning_rate=0.0001):
 
    # Encodeur
    input_layer = layers.Input(shape=(input_dim,))
    encoded = layers.Dense(n*128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(input_layer)
    encoded = layers.Dense(n*64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(encoded)
    encoded = layers.Dense(n*32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(encoded)


    latent_space = layers.Dense(n*16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(encoded)

    # Décodeur
    latent_input = layers.Input(shape=(n*16,))
    decoded = layers.Dense(n*32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(latent_input)
    decoded = layers.Dense(n*64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(decoded)
    decoded = layers.Dense(n*128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(pregu))(decoded)
    output_layer = layers.Dense(input_dim, activation='linear')(decoded)

    # Modèles
    encoder = models.Model(input_layer, latent_space, name="encoder")
    decoder = models.Model(latent_input, output_layer, name="decoder")
    autoencoder = models.Model(input_layer, decoder(encoder(input_layer)), name="autoencoder")

    # Compilation
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate)
    autoencoder.compile(optimizer=optimizer, loss=perte)

    return autoencoder, encoder, decoder

In [ ]:
#PERTE PERSONNALISE POUR L'AUTOENCODER
def custom_loss(weight_points=5.0):
    def loss_function(y_true, y_pred):
        
          

            # Calcul des différentes pertes
        points_loss = tf.reduce_mean(
                tf.square(y_true-y_pred
            ))

           
            
            # Ajout au total
        total_loss = points_loss*weight_points
        
        return total_loss

    return loss_function


In [ ]:
#ENTRAINE L'AUTOENCODER ET GARDE LE MEILLEUR MODELE ET AFFICHE LES COURBES DE LOSS

def train_autoencoder(cp, epochs,n,pregu,learning_rate,perte='mean_squared_loss'):
    cp=np.array(cp)
    # Normalisation des données
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(cp)
    input_dim = X_normalized.shape[1]
    print(input_dim)
    # Création de l'autoencodeur
    autoencoder, encoder, decoder = create_autoencoder(input_dim,perte=perte,n=n,pregu=pregu,learning_rate=learning_rate)
    
    # Division des données en ensemble d'entraînement et de validation
    X_train, X_val = train_test_split(X_normalized, test_size=0.2, random_state=42)
    
    # Callback pour sauvegarder le meilleur modèle
    checkpoint = ModelCheckpoint(
        'best_autoencoder.keras', 
        monitor='val_loss', 
        save_best_only=True, 
        mode='min', 
        verbose=0
    )
    
    # Entraînement
    history = autoencoder.fit(
        X_train,
        X_train,
        epochs=epochs,
        batch_size=32,
        validation_data=(X_val, X_val),
        callbacks=[checkpoint],
        verbose=0
    )
    
    # Charger le meilleur modèle
    autoencoder.load_weights('best_autoencoder.keras')
    
    # Tracer les courbes de perte
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Loss (Training)')
    plt.plot(history.history['val_loss'], label='Loss (Validation)')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Squared Error')
    plt.yscale('log')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()
    
    return autoencoder, encoder, decoder, X_val, scaler

In [ ]:
#AFFICHE LES TRAJECTOIRES RECONSTRUITES DU SET DE VALIDATION ET CALCULE L'ERREUR ASSOCIEE
def reconstruct_bez(autoencoder, X_val, scaler):
    X_reconstructed = autoencoder.predict(X_val)
    X_normalized = scaler.inverse_transform(X_reconstructed)
    X_original = scaler.inverse_transform(X_val)

    original1=[]
    reconstructed1=[]
    # Trajectoires originales et reconstruites
    all_trajectoire_original = []
    all_trajectoire_reconstructed = []

    for original, reconstructed in zip(X_original, X_normalized):
        traj_original, traj_reconstructed = [], []
        print(len(original))
        # Gestion selon la taille de la liste
        indices = list(range(8))
        step=8
        # Extraction et reconstruction
        for k in range(0, len(original), step):
            orig_packet = [original[k + i] for i in indices if k + i < len(original)]
            rec_packet = [reconstructed[k + i] for i in indices if k + i < len(reconstructed)]
            if k > 0 and len(orig_packet) == len(indices):
                orig_packet[:2] = traj_original[-1][-1]  # Dernière courbe de la trajectoire précédente
                rec_packet[:2] = traj_reconstructed[-1][-1]  # Idem pour la trajectoire reconstruite
            if len(orig_packet) == len(indices):
                # Crée les points de Bézier
                bez_orig = [orig_packet[j:j + 2] for j in range(0, len(orig_packet), 2)]
                bez_rec = [rec_packet[j:j + 2] for j in range(0, len(rec_packet), 2)]
                
                traj_original.append(bez_orig)
                traj_reconstructed.append(bez_rec)

        all_trajectoire_original.append(traj_original)
        all_trajectoire_reconstructed.append(traj_reconstructed)

    # Calcul de l'erreur L2
    plt.figure(figsize=(10,6))
    for traj_original, traj_reconstructed in zip(all_trajectoire_original, all_trajectoire_reconstructed):
        original_traj = []
        reconstructed_traj = []
        for curve_original, curve_reconstructed in zip(traj_original, traj_reconstructed):
            t_values = np.linspace(0, 1,1000)

            # Calcul des points sur les courbes originales et reconstruites
            bezier_points_original = np.array([bezier_q(curve_original, t) for t in t_values])
            bezier_points_reconstructed = np.array([bezier_q(curve_reconstructed, t) for t in t_values])

            # Accumulation des points
            original_traj.append(bezier_points_original)
            reconstructed_traj.append(bezier_points_reconstructed)
        
    # Calcul de l'erreur L2 globale
        original1.append(original_traj)
        reconstructed1.append(reconstructed_traj)
    
    l2_error=np.linalg.norm(np.array(original1)-np.array(reconstructed1),axis=1).mean()
    i=0
    for traj_original, traj_reconstructed in zip(all_trajectoire_original, all_trajectoire_reconstructed):
        for curve_original, curve_reconstructed in zip(traj_original, traj_reconstructed):
            curve_original = np.array(curve_original)
            curve_reconstructed = np.array(curve_reconstructed)

            t_values = np.linspace(0, 1, 1000)
            bezier_points_original = np.array([bezier_q(curve_original, t) for t in t_values])
            bezier_points_reconstructed = np.array([bezier_q(curve_reconstructed, t) for t in t_values])

            plt.plot(bezier_points_original[:, 0], bezier_points_original[:, 1], color='b', label='Original', alpha=0.7)
            plt.plot(bezier_points_reconstructed[:, 0], bezier_points_reconstructed[:, 1], color='r', label='Reconstruit', alpha=0.7)
        i+=1
        if i  > 4:
            break
    plt.xlabel('LONGITUDE')
    plt.ylabel('LATITUDE')
    plt.axis("equal")
    plt.show()
    print(f"Erreur moyenne L2 : {l2_error}")
    return l2_error

In [ ]:
#SAUVAGERDE LES MODELES D'AUTOENCODER,ENCODER ET DECODER

@register_keras_serializable()
def loss_function(weight_points=10.0):
    def loss_function(y_true, y_pred):
        
          

            # Calcul des différentes pertes
        points_loss = tf.reduce_mean(
                tf.square(y_true-y_pred
            ))

           
            
            # Ajout au total
        total_loss = points_loss*weight_points
        
        return total_loss

    return loss_function


save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

# Sauvegarder l'autoencodeur complet
autoencoder.save(os.path.join(save_dir, "autoencoder4.keras"))

# Sauvegarder les parties encoder et decoder individuellement
encoder.save(os.path.join(save_dir, "encoder4.keras"))
decoder.save(os.path.join(save_dir, "decoder4.keras"))

# Sauvegarder les données transformées X_valt et scaler
with open(os.path.join(save_dir, "X_val4.pkl"), "wb") as f:
    pickle.dump(X_val, f)

with open(os.path.join(save_dir, "scaler4.pkl"), "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
#CHARGE LES MODELES
autoencoder = load_model(os.path.join(save_dir, "autoencoder4.keras"))
encoder = load_model(os.path.join(save_dir, "encoder4.keras"))
decoder = load_model(os.path.join(save_dir, "decoder4.keras"))

with open(os.path.join(save_dir, "X_val4.pkl"), "rb") as f:
    X_val = pickle.load(f)

with open(os.path.join(save_dir, "scaler4.pkl"), "rb") as f:
    scaler = pickle.load(f)


In [ ]:
#CLUSTERISE LES TRAJECTOIRES AVEC BICRCH
def cluster_birch(df, premiers_points, points_barycentres, barycentres, n_clusters):
    
    features = np.hstack([premiers_points, points_barycentres])
    
    
    birch = Birch(threshold=0.5,n_clusters=n_clusters) 
    birch.fit(features)
    cluster_labels = birch.labels_  
    
    
    barycentres['Cluster'] = cluster_labels

    df_sorted = df.sort_values(by=['SYST_TRAJ_ID', 'SYST_POINT_ID'])
    df_clusters = df_sorted.merge(barycentres[['SYST_TRAJ_ID', 'Cluster']], on='SYST_TRAJ_ID', how='left')

 
    cmap = plt.get_cmap('tab10', n_clusters)  
    cluster_colors = {cluster: cmap(cluster) for cluster in range(n_clusters)}

    
    plt.figure(figsize=(12, 8))
    for cluster in range(n_clusters):
        cluster_data = df_clusters[df_clusters['Cluster'] == cluster]
        for traj_id, traj_points in cluster_data.groupby('SYST_TRAJ_ID'):
            plt.plot(
                traj_points['LONGITUDE'], 
                traj_points['LATITUDE'], 
                color=cluster_colors[cluster],  
                label=f"Cluster {cluster}" if traj_id == cluster_data['SYST_TRAJ_ID'].iloc[0] else "", 
                alpha=0.6
            )



    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Trajectoires complètes avec Clustering (Birch)')
    plt.legend()
    plt.grid(True)
    plt.show()
    print(n_clusters)
    return df_clusters,cluster_colors

premiers_points = resampled_df_curve_length.loc[resampled_df_curve_length.groupby('SYST_TRAJ_ID')['SYST_POINT_ID'].idxmin()].sort_values(by='SYST_TRAJ_ID')[['LONGITUDE', 'LATITUDE']].to_numpy()
derniers_points = resampled_df_curve_length.sort_values(by=['SYST_TRAJ_ID', 'SYST_POINT_ID']).groupby('SYST_TRAJ_ID').tail(15)
barycentres = derniers_points.groupby('SYST_TRAJ_ID')[['LATITUDE', 'LONGITUDE']].mean().reset_index()
points_barycentres = barycentres[['LONGITUDE', 'LATITUDE']].to_numpy()
syst_traj_ids = barycentres['SYST_TRAJ_ID'].to_numpy()

df_clusters,cluster_colors=cluster_birch(
    df=resampled_df_curve_length, 
    premiers_points=premiers_points, 
    points_barycentres=points_barycentres, 
    barycentres=barycentres, 
    n_clusters=4)

In [ ]:
#ASSSOCIE A CHAQUE ID DE TRAJ UN CLUSTER
id_cluster_mapping = (
    df_clusters.groupby('SYST_TRAJ_ID', sort=True)['Cluster']
    .first()
    .reset_index()
)

In [ ]:
#PROJETTE L'ESPACE LATENT EN 2D

cp = np.array(cp)
X_normalized = scaler.fit_transform(cp)
X_latent = encoder.predict(X_normalized)


reducer = umap.UMAP(n_components=2, n_neighbors=50, min_dist=0.01, metric='euclidean', random_state=42)
X_umap = reducer.fit_transform(X_latent)

df_umap = pd.DataFrame(X_umap, columns=['Dim1', 'Dim2'])
df_umap['Cluster'] = id_cluster_mapping['Cluster'].values

plt.figure(figsize=(10, 7))
scatter = plt.scatter(
    df_umap['Dim1'],
    df_umap['Dim2'],
    c=df_umap['Cluster'].map(lambda cluster: cluster_colors[cluster]),
    s=50,
    alpha=0.7
)

# Ajout des légendes
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Projection UMAP colorée par Cluster (paramètres optimaux)')
plt.grid(True)
plt.show()


# t-SNE
tsne = TSNE(n_components=2, perplexity=50, learning_rate=10, n_iter=1000, random_state=42)
X_tsne = tsne.fit_transform(X_latent)

df_tsne = pd.DataFrame(X_tsne, columns=['Dim1', 'Dim2'])
df_tsne['Cluster'] = id_cluster_mapping['Cluster'].values

# Visualisation des résultats t-SNE
plt.figure(figsize=(10, 7))
plt.scatter(
    df_tsne['Dim1'],
    df_tsne['Dim2'],
    c=df_tsne['Cluster'].map(lambda cluster: cluster_colors[cluster]),
    s=50,
    alpha=0.7
)
plt.title('Projection t-SNE (paramètres optimisés)')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid(True)
plt.show()


In [ ]:
# Fonction pour optimiser les paramètres de DBSCAN
def optimize_dbscan(X, eps_values, min_samples_values):
    best_params = None
    best_score = -1
    results = []

    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(X)

            # Filtrer les outliers (label = -1)
            if len(set(labels)) > 1 and -1 not in set(labels):
                score = silhouette_score(X, labels)
                results.append((eps, min_samples, score))
                if score > best_score:
                    best_score = score
                    best_params = (eps, min_samples)

    return best_params, best_score, results

# Paramètres à tester pour DBSCAN
eps_values = [5.0]
min_samples_values = [20,30,40,50]

# Optimisation de DBSCAN pour UMAP
best_params_umap, best_score_umap, dbscan_results_umap = optimize_dbscan(X_umap, eps_values, min_samples_values)
print(f"Meilleurs paramètres DBSCAN (UMAP): eps={best_params_umap[0]}, min_samples={best_params_umap[1]} avec un score silhouette de {best_score_umap:.4f}")

# Clustering DBSCAN avec les meilleurs paramètres pour UMAP
dbscan_umap = DBSCAN(eps=best_params_umap[0], min_samples=best_params_umap[1])
labels_umap = dbscan_umap.fit_predict(X_umap)

# Optimisation de DBSCAN pour t-SNE
best_params_tsne, best_score_tsne, dbscan_results_tsne = optimize_dbscan(X_tsne, eps_values, min_samples_values)
print(f"Meilleurs paramètres DBSCAN (t-SNE): eps={best_params_tsne[0]}, min_samples={best_params_tsne[1]} avec un score silhouette de {best_score_tsne:.4f}")

# Clustering DBSCAN avec les meilleurs paramètres pour t-SNE
dbscan_tsne = DBSCAN(eps=best_params_tsne[0], min_samples=best_params_tsne[1])
labels_tsne = dbscan_tsne.fit_predict(X_tsne)

# Génération du dictionnaire des couleurs
def generate_cluster_colors(labels, cmap_name='tab20'):
    unique_labels = set(labels)
    cmap = plt.get_cmap(cmap_name, len(unique_labels))
    cluster_colors = {label: cmap(i) for i, label in enumerate(sorted(unique_labels))}
    return cluster_colors

# Génération des couleurs pour UMAP et t-SNE
umap_colors = generate_cluster_colors(labels_umap)
tsne_colors = generate_cluster_colors(labels_tsne)

# Visualisation des clusters UMAP avec couleurs
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(
    df_umap['Dim1'],
    df_umap['Dim2'],
    c=[umap_colors[label] for label in labels_umap],
    s=50,
    alpha=0.7
)
plt.title('DBSCAN Clustering (UMAP)')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')

# Visualisation des clusters t-SNE avec couleurs
plt.subplot(1, 2, 2)
plt.scatter(
    df_tsne['Dim1'],
    df_tsne['Dim2'],
    c=[tsne_colors[label] for label in labels_tsne],
    s=50,
    alpha=0.7
)
plt.title('DBSCAN Clustering (t-SNE)')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')

plt.tight_layout()
plt.show()

# Création des DataFrames finaux contenant les clusters et indices des trajectoires
df_umap_clusters = pd.DataFrame({
    'Latent_Index': range(len(X_umap)),
    'UMAP_Cluster': labels_umap
})

df_tsne_clusters = pd.DataFrame({
    'Latent_Index': range(len(X_tsne)),
    'tSNE_Cluster': labels_tsne
})

# Affichage des DataFrames finaux contenant les clusters
print("Clusters DBSCAN pour UMAP :")
print(df_umap_clusters.head())

print("\nClusters DBSCAN pour t-SNE :")
print(df_tsne_clusters.head())

# Enregistrement des DataFrames dans des fichiers CSV pour les consulter ultérieurement
df_umap_clusters.to_csv("umap_clusters.csv", index=False)
df_tsne_clusters.to_csv("tsne_clusters.csv", index=False)

print("\nLes résultats ont été enregistrés dans 'umap_clusters.csv' et 'tsne_clusters.csv'.")

In [ ]:
def plot_trajectories_umap_tsne(df, traj_labels, col_traj_id, col_x, col_y, cluster_colors, title="Trajectoires colorées par cluster"):

    # Trier le DataFrame par l'ID des trajectoires
    df_sorted = df.sort_values(by=['SYST_TRAJ_ID', 'SYST_POINT_ID'])

    # Extraire les IDs de trajectoire uniques
    unique_traj_ids = df_sorted[col_traj_id].unique()

    # Vérifier que la taille des labels correspond aux IDs uniques
    if len(traj_labels) != len(unique_traj_ids):
        raise ValueError(f"Le nombre de labels ({len(traj_labels)}) ne correspond pas au nombre d'IDs uniques ({len(unique_traj_ids)}).")

    # Créer un DataFrame des clusters pour lier les labels
    cluster_mapping = pd.DataFrame({
        col_traj_id: unique_traj_ids,
        'Cluster': traj_labels
    })

    # Associer les clusters au DataFrame principal
    df_with_clusters = df_sorted.merge(cluster_mapping, on=col_traj_id, how='left')

    # Affichage des trajectoires
    plt.figure(figsize=(12, 8))
    for cluster in sorted(df_with_clusters['Cluster'].unique()):
        cluster_data = df_with_clusters[df_with_clusters['Cluster'] == cluster]
        color = cluster_colors.get(cluster, 'gray')  # Couleur définie pour chaque cluster ou gris pour les outliers
        for traj_id, traj_points in cluster_data.groupby(col_traj_id):
            plt.plot(
                traj_points[col_x],
                traj_points[col_y],
                color=color,
                label=f"Cluster {cluster}" if traj_id == cluster_data[col_traj_id].iloc[0] else "",
                alpha=0.6,
                lw=1.5
            )

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

    return df_with_clusters

In [ ]:
df_with_clusters = plot_trajectories_umap_tsne(
    df=resampled_df_curve_length,
    traj_labels=labels_umap,
    col_traj_id='SYST_TRAJ_ID',
    col_x='LONGITUDE',
    col_y='LATITUDE',
    cluster_colors=umap_colors,
    title="Trajectoires colorées par clusters (UMAP + DBSCAN)"
)

In [ ]:
def plot_trajectories_by_cluster_in_subplots(df, traj_labels, col_traj_id, col_x, col_y, cluster_colors, output_path=None):

    # Sort DataFrame for smoother trajectory plotting
    df_sorted = df.sort_values(by=['SYST_TRAJ_ID', 'SYST_POINT_ID'])

    # Get unique trajectory IDs
    unique_traj_ids = df_sorted[col_traj_id].unique()

    # Check consistency between labels and unique trajectory IDs
    if len(traj_labels) != len(unique_traj_ids):
        raise ValueError(f"The number of labels ({len(traj_labels)}) does not match the number of unique trajectory IDs ({len(unique_traj_ids)}).")

    # Map cluster labels to trajectory IDs
    cluster_mapping = pd.DataFrame({
        col_traj_id: unique_traj_ids,
        'Cluster': traj_labels
    })

    # Merge cluster information with the main DataFrame
    df_with_clusters = df_sorted.merge(cluster_mapping, on=col_traj_id, how='left')

    # Prepare clusters and subplots
    clusters = sorted(df_with_clusters['Cluster'].unique())
    n_clusters = len(clusters)
    n_cols = 3  # Number of columns in subplots
    n_rows = ceil(n_clusters / n_cols)  # Calculate rows based on number of clusters

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5), constrained_layout=True)
    axes = axes.flatten()  # Flatten axes for easier iteration

    for i, cluster in enumerate(clusters):
        ax = axes[i]
        cluster_data = df_with_clusters[df_with_clusters['Cluster'] == cluster]
        color = cluster_colors.get(cluster, 'gray')

        for traj_id, traj_points in cluster_data.groupby(col_traj_id):
            ax.plot(
                traj_points[col_x],
                traj_points[col_y],
                color=color,
                alpha=0.5,
                lw=0.8
            )

        # Subplot settings
        ax.set_title(f'Cluster {cluster}')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        ax.grid(True)

    # Hide empty subplots (if any)
    for j in range(len(clusters), len(axes)):
        fig.delaxes(axes[j])

    # Add a global legend
    handles = [plt.Line2D([], [], color=cluster_colors[cluster], label=f'Cluster {cluster}')
               for cluster in clusters]
    fig.legend(handles=handles, title="Clusters", loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=n_cols)

    # Save or display the figure
    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
    else:
        plt.show()
        

In [ ]:
plot_trajectories_by_cluster_in_subplots(
    df=resampled_df_curve_length,
    traj_labels=labels_umap,
    col_traj_id='SYST_TRAJ_ID',
    col_x='LONGITUDE',
    col_y='LATITUDE',
    cluster_colors=umap_colors,
    output_path='trajectories_by_cluster_subplots.png'
)

In [ ]:

### Réduction de dimension avec PCA ###
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_latent)

### Réduction de dimension avec t-SNE ###


# Création des DataFrames pour chaque méthode
df_pca = pd.DataFrame(X_pca, columns=['Dim1', 'Dim2'])
df_pca['Cluster'] = id_cluster_mapping['Cluster'].values



### Visualisation PCA ###
plt.figure(figsize=(10, 7))
scatter_pca = plt.scatter(
    df_pca['Dim1'], 
    df_pca['Dim2'], 
    c=df_pca['Cluster'].map(lambda cluster: cluster_colors[cluster]),  # Couleur selon le cluster,         # Colormap discrète pour les clusters
    s=50,
    alpha=0.7
)


# Configuration des axes et affichage
plt.title("Projection PCA")
plt.legend()
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Projection PCA colorée par Cluster')
plt.grid(True)
plt.show()



In [ ]:
#VISUALISER LES TRAJECTOIRES RECONSTRUITES PAR ID POUR INVESTIGUER LES ERREUR DE RECONSTRUCTIONS
def reconstruct_bez_id(cp, X_val, scaler, selected_ids):
    print(len(X_val))
    # Sélectionner uniquement les trajectoires correspondant aux indices donnés
    X_val_selected = X_val[selected_ids]
    
    # Reconstruction avec l'autoencodeur
    X_normalized = scaler.inverse_transform(X_val_selected)
    X_original = cp[selected_ids]

    original1 = []
    reconstructed1 = []
    all_trajectoire_original = []
    all_trajectoire_reconstructed = []

    for original, reconstructed in zip(X_original, X_normalized):
        traj_original, traj_reconstructed = [], []
        indices = list(range(8))
        step = 8

        for k in range(0, len(original), step):
            orig_packet = [original[k + i] for i in indices if k + i < len(original)]
            rec_packet = [reconstructed[k + i] for i in indices if k + i < len(reconstructed)]
            
            # Remplacer les deux premiers éléments du paquet par les deux précédents, si possible
            if k > 0 and len(orig_packet) == len(indices):
                orig_packet[:2] = traj_original[-1][-1]  # Dernière courbe de la trajectoire précédente
                rec_packet[:2] = traj_reconstructed[-1][-1]  # Idem pour la trajectoire reconstruite

            if len(orig_packet) == len(indices):
                bez_orig = [orig_packet[j:j + 2] for j in range(0, len(orig_packet), 2)]
                bez_rec = [rec_packet[j:j + 2] for j in range(0, len(rec_packet), 2)]
                
                traj_original.append(bez_orig)
                traj_reconstructed.append(bez_rec)

        all_trajectoire_original.append(traj_original)
        all_trajectoire_reconstructed.append(traj_reconstructed)

    plt.figure(figsize=(10, 6))
    for traj_original, traj_reconstructed in zip(all_trajectoire_original, all_trajectoire_reconstructed):
        original_traj = []
        reconstructed_traj = []
        for curve_original, curve_reconstructed in zip(traj_original, traj_reconstructed):
            t_values = np.linspace(0, 1, 1000)

            bezier_points_original = np.array([bezier_q(curve_original, t) for t in t_values])
            bezier_points_reconstructed = np.array([bezier_q(curve_reconstructed, t) for t in t_values])

            original_traj.append(bezier_points_original)
            reconstructed_traj.append(bezier_points_reconstructed)

        original1.append(original_traj)
        reconstructed1.append(reconstructed_traj)
        
    l2_error = np.linalg.norm(np.array(original1) - np.array(reconstructed1), axis=1).mean()
    
    # Tracé des trajectoires sélectionnées
    for traj_original, traj_reconstructed in zip(all_trajectoire_original, all_trajectoire_reconstructed):
        for curve_original, curve_reconstructed in zip(traj_original, traj_reconstructed):
            curve_original = np.array(curve_original)
            curve_reconstructed = np.array(curve_reconstructed)

            t_values = np.linspace(0, 1, 1000)
            bezier_points_original = np.array([bezier_q(curve_original, t) for t in t_values])
            bezier_points_reconstructed = np.array([bezier_q(curve_reconstructed, t) for t in t_values])

            plt.plot(bezier_points_original[:, 0], bezier_points_original[:, 1], color='b', label='Original', alpha=0.7)
            plt.plot(bezier_points_reconstructed[:, 0], bezier_points_reconstructed[:, 1], color='r', label='Reconstruit', alpha=0.7)

    plt.xlabel('LONGITUDE')
    plt.ylabel('LATITUDE')
    plt.axis("equal")
    plt.title("Trajectoires sélectionnées : Originales vs Reconstruites")
    plt.legend(['Original', 'Reconstruit'])
    plt.grid(True)
    plt.show()

    print(f"Erreur moyenne L2 pour les trajectoires sélectionnées : {l2_error}")
    return l2_error
X_decoded=decoder.predict(X_latent)
reconstruct_bez_id(cp,autoencoder, X_decoded,scaler,selected_ids=[695])

In [ ]:
#CALCULE LES ERREURS POUR CHAQUE TRAJECTOIRE

def errors_df(cp, X_decoded, scaler):
    X_original = cp
    X_decoded = scaler.inverse_transform(X_decoded)
    

    errors = []
    indices = []

    for idx, (original, reconstructed) in enumerate(zip(X_original, X_decoded)):
        traj_original, traj_reconstructed = [], []
        step = 8  # Taille de chaque courbe Bézier (4 points de contrôle)

        for k in range(0, len(original), step):
            orig_packet = original[k:k + step]
            rec_packet = reconstructed[k:k + step]
            if k > 0 and len(orig_packet) == len(indices):
                orig_packet[:2] = traj_original[-1][-1]  # Dernière courbe de la trajectoire précédente
                rec_packet[:2] = traj_reconstructed[-1][-1]  # Idem pour la trajectoire reconstruite

            if len(orig_packet) == step:
                bez_orig = [orig_packet[j:j + 2] for j in range(0, len(orig_packet), 2)]
                bez_rec = [rec_packet[j:j + 2] for j in range(0, len(rec_packet), 2)]

                traj_original.append(bez_orig)
                traj_reconstructed.append(bez_rec)

        # Calcul des points sur les courbes et de l'erreur pour cette trajectoire
        t_values = np.linspace(0, 1, 1000)
        original_points = np.concatenate(
            [np.array([bezier_q(curve, t) for t in t_values]) for curve in traj_original], axis=0
        )
        reconstructed_points = np.concatenate(
            [np.array([bezier_q(curve, t) for t in t_values]) for curve in traj_reconstructed], axis=0
        )
        
        # Calcul de l'erreur moyenne L2 pour la trajectoire
        l2_error = np.linalg.norm(np.array(original_points)- np.array(reconstructed_points), axis=1).mean()

        errors.append(l2_error)
        indices.append(idx)

    # Création du DataFrame avec les indices et les erreurs
    error_df = pd.DataFrame({
        'Latent_Index': indices,
        'Reconstruction_Error': errors
    })

    return error_df
error_df=errors_df(cp,X_decoded,scaler)
# Fusion avec les clusters UMAP
df_umap_clusters_with_errors = pd.merge(df_umap_clusters, error_df, on='Latent_Index', how='left')

# Fusion avec les clusters t-SNE
df_tsne_clusters_with_errors = pd.merge(df_tsne_clusters, error_df, on='Latent_Index', how='left')

In [ ]:
# Moyenne des erreurs par cluster (UMAP)
umap_cluster_errors = df_umap_clusters_with_errors.groupby('UMAP_Cluster')['Reconstruction_Error'].mean().reset_index()
umap_cluster_errors = umap_cluster_errors.rename(columns={'Reconstruction_Error': 'Mean_Error'})

# Moyenne des erreurs par cluster (t-SNE)
tsne_cluster_errors = df_tsne_clusters_with_errors.groupby('tSNE_Cluster')['Reconstruction_Error'].mean().reset_index()
tsne_cluster_errors = tsne_cluster_errors.rename(columns={'Reconstruction_Error': 'Mean_Error'})

# Affichage des erreurs moyennes par cluster
print("Erreurs moyennes par cluster (UMAP) :")
print(umap_cluster_errors)

print("\nErreurs moyennes par cluster (t-SNE) :")
print(tsne_cluster_errors)


In [ ]:
# Visualisation des erreurs moyennes par cluster (UMAP)
plt.figure(figsize=(10, 5))
plt.bar(umap_cluster_errors['UMAP_Cluster'], umap_cluster_errors['Mean_Error'])
plt.title('Erreurs moyennes de reconstruction par cluster (UMAP)')
plt.xlabel('Cluster UMAP')
plt.ylabel('Erreur moyenne')
plt.xticks(rotation=45)
plt.grid()
plt.show()

# Visualisation des erreurs moyennes par cluster (t-SNE)
plt.figure(figsize=(10, 5))
plt.bar(tsne_cluster_errors['tSNE_Cluster'], tsne_cluster_errors['Mean_Error'])
plt.title('Erreurs moyennes de reconstruction par cluster (t-SNE)')
plt.xlabel('Cluster t-SNE')
plt.ylabel('Erreur moyenne')
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
#VISUALISATION DES ERREURS INTRA CLUSTER

# Obtenir les clusters uniques et trier
clusters = sorted(df_umap_clusters_with_errors['UMAP_Cluster'].unique())

# Déterminer le nombre de colonnes et de lignes pour les subplots
n_clusters = len(clusters)
n_cols = 3  # Nombre de colonnes
n_rows = (n_clusters + n_cols - 1) // n_cols  # Calcul du nombre de lignes nécessaires

# Initialiser la figure et les subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5), constrained_layout=True)
axes = axes.flatten()  # Aplatir les axes pour un parcours plus facile

# Générer un histogramme pour chaque cluster
for i, cluster in enumerate(clusters):
    cluster_data = df_umap_clusters_with_errors[df_umap_clusters_with_errors['UMAP_Cluster'] == cluster]
    
    # Tracer l'histogramme dans le subplot correspondant
    axes[i].hist(cluster_data['Reconstruction_Error'], bins=30, color='blue', alpha=0.7, edgecolor='black')
    axes[i].set_title(f"Cluster {cluster}")
    axes[i].set_xlabel("Reconstruction Error")
    axes[i].set_ylabel("Frequency")
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

# Supprimer les subplots vides si le nombre de clusters est inférieur au nombre de subplots
for j in range(len(clusters), len(axes)):
    fig.delaxes(axes[j])

# Ajouter un titre global pour la figure
fig.suptitle("Distribution of Reconstruction Errors by Cluster", fontsize=16)

# Sauvegarder la figure complète
plt.savefig("all_clusters_errors.png", dpi=300)
plt.show()

print("Les histogrammes combinés ont été générés et sauvegardés.")

In [ ]:
#DETECTE LES TRAJECTOIRES AVEC UNE ERREUR DE RECONSTRUCTION TROP GRANDE

thresholds = df_umap_clusters_with_errors.groupby('UMAP_Cluster')['Reconstruction_Error'].quantile(0.99)

# Ajouter une colonne indiquant si la trajectoire est un outlier
df_umap_clusters_with_errors['Is_Outlier'] = df_umap_clusters_with_errors.apply(
    lambda row: row['Reconstruction_Error'] > thresholds[row['UMAP_Cluster']],
    axis=1
)

# Obtenir les outliers par cluster
outliers_by_cluster = {
    cluster: df_umap_clusters_with_errors[
        (df_umap_clusters_with_errors['UMAP_Cluster'] == cluster) &
        (df_umap_clusters_with_errors['Is_Outlier'])
    ]['Latent_Index'].tolist()
    for cluster in thresholds.index
}

# Afficher les seuils par cluster
print("Seuils (99e quantile) par cluster :")
print(thresholds)

# Afficher les outliers par cluster
print("\nOutliers par cluster :")
for cluster, outliers in outliers_by_cluster.items():
    print(f"Cluster {cluster}: {outliers}")

In [ ]:
df_umap_clusters['Dim1'] = df_umap['Dim1']
df_umap_clusters['Dim2'] = df_umap['Dim2']

In [ ]:
#AFFCIHE LES OUTLIERS POUR UMAP

outlier_indices=[668, 589, 528, 542, 66, 527, 377, 986, 978, 210, 120, 174, 247, 973, 914]


df_umap_clusters['Is_Outlier'] = df_umap_clusters['Latent_Index'].isin(outlier_indices)
plt.figure(figsize=(10, 6))
plt.scatter(
    df_umap_clusters['Dim1'],
    df_umap_clusters['Dim2'],
    c=[umap_colors[label] if not is_outlier else 'black' for label, is_outlier in zip(df_umap_clusters['UMAP_Cluster'], df_umap_clusters['Is_Outlier'])],
    s=50,
    alpha=[0.06 if not is_outlier else 1.0 for is_outlier in df_umap_clusters['Is_Outlier']]  # Transparence pour les non-outliers

)
plt.title('DBSCAN Clustering (UMAP) avec Outliers')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid(True)
plt.show()

In [ ]:
#AFFCIHE LES OUTLIERS POUR PCA

highlight_indices = [668, 589, 528, 542, 66, 527, 377, 986, 978, 210, 120, 174, 247, 973, 914]

df_pca['Highlight'] = df_pca.index.isin(highlight_indices)

# Scatter plot
plt.figure(figsize=(10, 7))
plt.scatter(
    df_pca['Dim1'], 
    df_pca['Dim2'], 
    c=[
        'black' if highlight else cluster_colors[cluster] 
        for highlight, cluster in zip(df_pca['Highlight'], df_pca['Cluster'])
    ],  # Couleur noire pour les points marqués
    s=[
        100 if highlight else 50 
        for highlight in df_pca['Highlight']
    ],  # Points plus grands pour les marqués
    alpha=0.7
)

# Configuration des axes et du titre
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Projection PCA avec points spécifiques en noir')
plt.grid(True)
plt.show()